In [1]:
from monitor import monitor
import sys
import traceback
import os
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime 
from fake_useragent import UserAgent

In [11]:
def url():
    pages_list = [30,31]
    urls_list = []
    for x in pages_list:
        education_url = f'https://www.sishutang.com.tw/course-list/Autodesk_%E8%A3%BD%E9%80%A0%E6%A5%AD2D%E7%B3%BB%E5%88%97%E8%AA%B2%E7%A8%8B/{x}'
        page = requests.get(education_url)
        soup = bs(page.text,'lxml')
        for i in range(1,30):
            href = soup.select(f"#course > section > div.content-wrap > div.content-detail > div > div.course > div > div:nth-child({i}) > div > div.text > a")
            for url in href:
                urls = url["href"]
                urls_list.append(urls)
    return urls_list
def main(urls_list):
    options = webdriver.ChromeOptions()
#     options.add_argument("--headless")
    driver = webdriver.Chrome(chrome_options = options, executable_path = "/Users/t3019ein/Desktop/chromedriver")
    courses = url()
    item_list = []
#   課程主頁的網址
    for j in range(len(courses)):
        print(j)
        page = driver.get(courses[j])
        items_a = driver.find_elements_by_xpath("//div[@class='info-head head-skin01']/h1[@class='info-title']")
        driver.implicitly_wait(3)
        dic={}
#       課程名稱
        for a in items_a:
            dic["title"] = a.text.strip().replace("\n","")
            items_b = driver.find_elements_by_xpath("//div[@class='course-lector']/div[@class='hour']/span")
            driver.implicitly_wait(3)
#       上課時數
        for b in items_b:
            dic["hours"] = b.text.strip().replace("(小時)","")
            items_c = driver.find_elements_by_xpath("//div[@class='course-lector']/div[@class='lector']/span")
            driver.implicitly_wait(3)
#       上課老師
        for c in items_c:
            dic["teacher"] = c.text.strip().replace("/","")
        items_d = driver.find_elements_by_xpath("//div[@class='recommend']/div[@class='price']")
        driver.implicitly_wait(3)
#       課程價格
        for d in items_d:
            dic["price"] = d.text.strip().replace("NT$","").replace("月","").replace("|","").replace("/","").replace(",","")
        today = datetime.today().strftime("%Y%m%d")
#       爬蟲時間
        dic["today"] = today
        item_list.append(dic)
        time.sleep(3)
    driver.quit()
    return item_list
def putintoSql(item_list):
#     dtype1={
#             'price':sqlalchemy.types.INTEGER(),
#             'hours':sqlalchemy.types.Float(precision=1),
#         }
    df = pd.DataFrame(item_list)
    engine = create_engine(f'mysql+pymysql://{loguser}:{logpw}@{logip}:3306/{logdb}?charset=utf8')
    df.to_sql('wash',engine,index=False)

In [9]:
url()
urls_list = url()

In [12]:
main(urls_list)

/Users/t3019ein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


[{'title': 'Autodesk AutoCAD 進階應用課程Autodesk MFG 2D AutoCAD 進階應用',
  'hours': '1.19 ',
  'teacher': ' 阮振宇 Alex Raun',
  'price': ' 800  ',
  'today': '20200723'},
 {'title': 'Autodesk AutoCAD Mechanical 入門課程Autodesk MFG 2D AutoCAD Mechanical 入門',
  'hours': '1.08 ',
  'teacher': ' 阮振宇 Alex Raun',
  'price': ' 500  ',
  'today': '20200723'},
 {'title': 'Autodesk AutoCAD Electrical 入門課程 第1回 | 共3回Autodesk MFG 2D AutoCAD Electrical 入門',
  'hours': '1.18 ',
  'teacher': ' 林盈呈Peter Lin',
  'price': ' 650  ',
  'today': '20200723'},
 {'title': 'Autodesk AutoCAD 基礎課程Autodesk MFG 2D AutoCAD 基礎',
  'hours': '1.24 ',
  'teacher': ' 阮振宇 Alex Raun',
  'price': ' 650  ',
  'today': '20200723'},
 {'title': 'Autodesk AutoCAD Electrical 入門課程 第2回 | 共3回',
  'hours': '1.5 ',
  'teacher': ' 林盈呈Peter Lin',
  'price': ' 650  ',
  'today': '20200723'},
 {'title': 'Autodesk AutoCAD Electrical 入門課程 第3回 | 共3回',
  'hours': '1.2 ',
  'teacher': ' 林盈呈Peter Lin',
  'price': ' 650  ',
  'today': '20200723'},
 {'title'